<a href="https://colab.research.google.com/github/Wildemberg-Projects/Plants-Diseases/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Instalação de Dependências
# Para utilização do Colab, modifique o '%' por '!'
# As abaixo comentadas são somente para instalar em ambiente local
# %pip install -q -U tensorflow
# %pip install -q -U keras
# %pip install -q -U numpy
# %pip install -q -U pandas
# %pip install -q -U Jinja2
# Uso da GPU precisa ser habilitado pelo comando a seguir
# %python3 -m pip install tensorflow[and-cuda]
%pip install -q -U tensorflow-addons
%pip install -q -U tensorflow-datasets
%pip install -q -U keras-utils
%pip install -q -U matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import tensorflow_datasets as tfds
import pandas as pd
from numpy import mean
import tensorflow as tf
import keras

In [43]:
# Coletando dados
tf.keras.backend.clear_session()
(ds_train), info = tfds.load('plant_village', split='train', with_info=True)

# Gerando um dataframe
df = tfds.as_dataframe(ds_train.take(20), info)

print(df.image, df.label)
# a estrutura de uma imagem (244,244,3), representa a largura e altura em pixels,
# e o 3 representa a quantidade de canais de cores pq é RGB

2024-04-20 11:49:03.070081: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


0     [[[171, 161, 169], [158, 148, 156], [153, 143,...
1     [[[168, 130, 119], [168, 130, 119], [167, 129,...
2     [[[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], ...
3     [[[193, 192, 200], [192, 191, 199], [189, 188,...
4     [[[96, 98, 121], [114, 116, 139], [99, 101, 12...
5     [[[173, 169, 160], [170, 166, 157], [166, 162,...
6     [[[200, 195, 199], [180, 175, 179], [204, 199,...
7     [[[202, 198, 199], [197, 193, 194], [216, 212,...
8     [[[116, 116, 108], [126, 126, 118], [131, 131,...
9     [[[142, 125, 118], [151, 134, 127], [141, 124,...
10    [[[115, 105, 104], [117, 107, 106], [119, 109,...
11    [[[173, 176, 195], [189, 192, 211], [169, 172,...
12    [[[113, 100, 153], [115, 102, 155], [116, 103,...
13    [[[115, 106, 111], [115, 106, 111], [118, 109,...
14    [[[85, 77, 98], [102, 94, 115], [126, 118, 139...
15    [[[109, 100, 103], [112, 103, 106], [122, 113,...
16    [[[111, 98, 105], [104, 91, 98], [107, 94, 101...
17    [[[160, 154, 166], [142, 136, 148], [143, 

In [44]:
# Dividir os dados de treinamento em treinamento e validação
# Embaralhando os dados para garantir que tanto a validação quanto o treinamento tenham todos os tipos de dados
ds_train = ds_train.shuffle(buffer_size=1000, seed=42)  

# Calcular o tamanho do conjunto de validação como 25% dos dados de treinamento
train_size = ds_train.cardinality().numpy()
validation_size = train_size // 4  # 25% dos dados de treinamento

# Dividindo os dados
ds_validation = ds_train.take(validation_size)
ds_train = ds_train.skip(validation_size)

# Verificar o número de exemplos em cada conjunto de dados
num_train_examples = ds_train.cardinality().numpy()
num_validation_examples = ds_validation.cardinality().numpy()
print("Número de exemplos de treinamento:", num_train_examples)
print("Número de exemplos de validação:", num_validation_examples)

Número de exemplos de treinamento: 40728
Número de exemplos de validação: 13575


In [47]:
# Modules used for data handling and visualisation
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random as r
sns.set_style("whitegrid")

# Modules used for suppressing warnings
import warnings
warnings.filterwarnings('ignore')

# Modules used for dataset split
import os
# Centering all the output images in the notebook.
from IPython.core.display import HTML as Center

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

# Returns both train and test split separately
PlantVillage: pd.DataFrame = tfds.as_dataframe(ds_train, info)
PlantVillage.name = 'PlantVillage'

from google.colab import data_table
data_table.enable_dataframe_formatter()

PlantVillage.head(10)
# Define the dictionary
label_mapping = {
  0: "Apple_scab",
  1: "Apple_black_rot",
  2: "Apple_cedar_apple_rust",
  3: "Apple_healthy",
  4: "Background_without_leaves",
  5: "Blueberry_healthy",
  6: "Cherry_powdery_mildew",
  7: "Cherry_healthy",
  8: "Corn_gray_leaf_spot",
  9: "Corn_common_rust",
  10: "Corn_northern_leaf_blight",
  11: "Corn_healthy",
  12: "Grape_black_rot",
  13: "Grape_black_measles",
  14: "Grape_leaf_blight",
  15: "Grape_healthy",
  16: "Orange_haunglongbing",
  17: "Peach_bacterial_spot",
  18: "Peach_healthy",
  19: "Pepper_bacterial_spot",
  20: "Pepper_healthy",
  21: "Potato_early_blight",
  22: "Potato_healthy",
  23: "Potato_late_blight",
  24: "Raspberry_healthy",
  25: "Soybean_healthy",
  26: "Squash_powdery_mildew",
  27: "Strawberry_healthy",
  28: "Strawberry_leaf_scorch",
  29: "Tomato_bacterial_spot",
  30: "Tomato_early_blight",
  31: "Tomato_healthy",
  32: "Tomato_late_blight",
  33: "Tomato_leaf_mold",
  34: "Tomato_septoria_leaf_spot",
  35: "Tomato_spider_mites_two_spotted_spider_mite",
  36: "Tomato_target_spot",
  37: "Tomato_mosaic_virus",
  38: "Tomato_yellow_leaf_curl_virus"
}

# Extract numerical labels from the DataFrame
labels = PlantVillage['label'].values

# Convert label names to their corresponding numerical codes
label_codes = [label_mapping[label] for label in labels]

# Convert label codes to a NumPy array
labels_array = np.array(label_codes)

ModuleNotFoundError: No module named 'seaborn'